In [19]:
# Cleaning the data from the database and integrating with Groq LLM
# This script is part of a larger system that manages parts data and provides search functionality.

import pandas as pd
import sqlite3

In [20]:
# Define the path to your database 
db_path = 'parts.db'
table = 'parts'

In [21]:
try : 
    conn = sqlite3.connect(db_path)
    print(f"Successfully connected")
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    conn.execute(""" UPDATE parts  
                 SET AS Category""")
    print(f"Data loaded successfully from {table} table.")
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()
        print("Connection closed.")

Successfully connected
An error occurred: near "AS": syntax error
Connection closed.


In [34]:
import pandas as pd
import sqlite3  

# Connect to your database
conn = sqlite3.connect('parts.db')  # update with your actual database

# Optional: run the update query first
conn.execute("""
    UPDATE parts
    SET category = 'brakes'
    WHERE category = 'Breaks';
""")
conn.commit() 



In [38]:
query = "SELECT * FROM parts;"
df = pd.read_sql_query(query, conn)
df.head()

,id,TransactionID,Date,CustomerID,PartNo,Quantity,Rate,TotalPrice,PartDescription,Category,Source,VehicleMake
0,0,4269,21-01-2023,CUST-342,SDT10193(STD),3,12.89,38.67,CLUTCHDISK,Others,Source,Honda
1,1,8603,22-06-2024,CUST-834,M748A-STD,2,8.97,17.94,LOWER ARM BUSH,Engine,Aftermarket,VehicleMake
2,2,5669,17-09-2024,CUST-829,KYB334698,1,4.72,4.72,ENGMATE,Others,OEM,Toyota
3,3,958,13-11-2023,CUST-162,04371-60070,1,5.46,5.46,ENGINE INSULATION,Others,OEM,Nissan
4,4,7117,05-10-2024,CUST-328,GWT131,1,5.95,5.95,SHOCK ABSORBER,Engine,Aftermarket,Mitsubishi


In [39]:
df['Category'].unique()

array(['Others', 'Engine', 'fuel', 'Body', 'brakes'], dtype=object)

In [32]:
categories = df['Category'].unique()
print(f"Unique categories found: {categories}")

Unique categories found: ['Others' 'Engine' 'fuel' 'Body' 'Breaks']


In [6]:
""" Cleaning the categories and classifying them into 5 main categories.
1.Engine
2.Body
3.Breaks
4.Fuel
5.Others
"""
# This dictionary maps each of your specific categories to a main one.
# It's more precise than the previous keyword-based approach.
exact_mapping = {
    # Engine
    'Engines & Systems': 'Engine',
    'Engine Parts': 'Engine',
    'engine Parts': 'Engine',
    'Engines & Engine Parts': 'Engine',
    'Engines & Drivetrain': 'Engine',
    'Engines & Cars': 'Engine',
    'Engine Mounts': 'Engine',
    'Cooling System': 'Engine',
    'Ignition': 'Engine',
    'Valves': 'Engine',
    'Gaskets & Seals': 'Engine',
    'Belts & Hoses': 'Engine',
    'Drivetrain': 'Engine',
    'Air Filtration': 'Engine',
    
    # Body
    'Suspension': 'Body',
    'Suspenders': 'Body', # Assumed typo for Suspension
    'Body & Trim': 'Body',
    
    # Fuel
    'Fuel Systems': 'fuel',
    'Fuel System': 'fuel',
    
    # Brakes
    'BrakingSystem': 'Brakes',
    'Braking System': 'Brakes',
    'Braking Systems': 'Brakes',
    
    # Others (Steering, Fluids, Filters, and miscellaneous)
    'Power Steoring': 'Others', # Typo for Steering
    'Steering': 'Others',
    ' steering': 'Others',
    'Fluids & Chemicals': 'Others',
    'Filtration': 'Others',
    'Filter': 'Others',
    'Estimated': 'Others',
    'Category': 'Others'
}

def classify_category(category: str) -> str:
    """
    Classifies a category using a direct mapping.
    Uses .strip() to handle potential extra spaces.
    """
    return exact_mapping.get(category.strip(), 'Others')

In [7]:
print("Updating the 'Category' column...")
df['Category'] = df['Category'].apply(classify_category)

Updating the 'Category' column...


In [8]:
categories = df['Category'].unique()
print(f"Unique categories found: {categories}")

Unique categories found: ['Others']


In [9]:
print(df.sample(50))

        id TransactionID        Date CustomerID         PartNo  Quantity  \
1775  1775            50  28-02-2023   CUST-265      M748A-STD         1   
815    815          2969  11-04-2023   CUST-534    83320‑0K01A         3   
492    492          2166  01-03-2025   CUST-505    21200-31U1B         2   
2938  2938           123  23-05-2023   CUST-429    90445-18130         1   
1848  1848           139  11-06-2023   CUST-361      KYB334399         2   
913    913          3291  24-03-2025   CUST-540    23826-31080         2   
2823  2823            96  22-04-2023   CUST-305    KYB33340187         2   
1373  1373             6  08-01-2023   CUST-489     DIAMBD-502         1   
1713  1713           127  28-05-2023   CUST-218  SDT10193(.50)         1   
1206  1206           104  01-05-2023   CUST-355    90311-T0016         1   
354    354          7840  11-11-2023   CUST-232    23300-75140         2   
1394  1394            62  14-03-2023   CUST-292    04495-06050         1   
1916  1916  

In [10]:
df.head(10)

,id,TransactionID,Date,CustomerID,PartNo,Quantity,Rate,TotalPrice,PartDescription,Category,Source,VehicleMake
0,0,4269,21-01-2023,CUST-342,SDT10193(STD),3,12.89,38.67,CLUTCHDISK,Others,Source,Honda
1,1,8603,22-06-2024,CUST-834,M748A-STD,2,8.97,17.94,LOWER ARM BUSH,Others,Aftermarket,VehicleMake
2,2,5669,17-09-2024,CUST-829,KYB334698,1,4.72,4.72,ENGMATE,Others,OEM,Toyota
3,3,958,13-11-2023,CUST-162,04371-60070,1,5.46,5.46,ENGINE INSULATION,Others,OEM,Nissan
4,4,7117,05-10-2024,CUST-328,GWT131,1,5.95,5.95,SHOCK ABSORBER,Others,Aftermarket,Mitsubishi
5,5,5262,04-08-2024,CUST-523,04495-06050,1,16.77,16.77,GROMMET,Others,Aftermarket,VehicleMake
6,6,2122,18-03-2025,CUST-516,SDT10193(STD),3,17.22,51.66,THRUST WASher,Others,Aftermarket,Nissan
7,7,4842,17-09-2024,CUST-438,DIAMBD-502,2,14.88,29.76,FUEL FILTER,Others,Aftermarket,Nissan
8,8,2847,09-02-2023,CUST-151,11011-38050,3,10.72,32.16,air filter,Others,Aftermarket,Generic
9,9,6858,06-11-2023,CUST-913,72-452,1,9.41,9.41,PETROLLOGGLE,Others,Source,Honda


In [11]:
df.to_csv('cleaned_parts.csv', index=False)


In [36]:
df["Category"=="Breaks"].sum()

KeyError: False